In [1]:
import parselmouth
import os.path
import glob

for wav_file in glob.glob("../data/*.wav"):
    print("processing {}...".format(wav_file))
    s = parselmouth.Sound(wav_file)
    s.pre_emphasize()
    s.save(os.path.splitext(wav_file)[0] + "_pre.wav", 'WAV')

processing ../data/sw02005_sph.wav...


In [2]:
import pandas as pd

print(pd.read_table("../praat-prosody_v0.1.1/demo/work_dir/pf_files/demo_C-pf.Tab"))

    WORD$                     WAV$ SPKR_ID$    GEN$  WORD_START  WORD_END  \
0    UMHM  ../demo/data/demo_C.wav        C  female         672       709   
1    OKAY  ../demo/data/demo_C.wav        C  female        1442      1464   
2    I'LL  ../demo/data/demo_C.wav        C  female        1464      1485   
3    I'LL  ../demo/data/demo_C.wav        C  female        1547      1557   
4   WRITE  ../demo/data/demo_C.wav        C  female        1557      1580   
5     YOU  ../demo/data/demo_C.wav        C  female        1608      1627   
6    TELL  ../demo/data/demo_C.wav        C  female        1630      1652   
7      ME  ../demo/data/demo_C.wav        C  female        1652      1668   
8     YOU  ../demo/data/demo_C.wav        C  female        1677      1689   
9    TELL  ../demo/data/demo_C.wav        C  female        1689      1714   
10     ME  ../demo/data/demo_C.wav        C  female        1714      1725   
11    YOU  ../demo/data/demo_C.wav        C  female        1725      1739   